## Regression basics

This notebook is about the basics of linear regression. In it we'll discuss: 
- creating a model, 
- how the OLS calculations work, 
- how to make a prediction,
- what residuals are, and
- how to measure the error (RMSE) of a model's prediction.

We'll also discuss $R^2$ as a measure of a model's explanatory power.

In [ ]:
# First, load the necessary libraries
import pandas as pd # hold data
import numpy as np  # math library


import plotly.express as px       # graphing library
import plotly.graph_objects as go # graphing library
import seaborn as sns             # graphing library

## Load data on cars to inspect

In [ ]:
data = sns.load_dataset('mpg') # load a built-in car dataframe
data.head() 

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


### Let's examine how displacement is related to fuel efficiency as measured by mpg (miles to the gallon of gas.)

Create a scatter plot to see if there are any obvious relationships between these two variables.

#### Trendline
Note the argument:
<pre>trendline="OLS"</pre>
This will overlay a trend line over the scatter plot.

What is the relationship between displacement (x-axis) and MPG (y-axis)?

In [ ]:
fig = px.scatter(data, x="displacement", y="mpg", trendline="ols")
fig

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Get the equation of the trend line from above.

We know that the a line can be represented by an equation like
<pre>
y = b + mx
</pre>
where **b** is the intercept of the line and **m** is the slope of the line.

Consider the output of the trendline calculated which was calculated using OLS (ordinary least squares) method.

In [ ]:
model = px.get_trendline_results(fig).iloc[0][0]
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.647
Model:                            OLS   Adj. R-squared:                  0.646
Method:                 Least Squares   F-statistic:                     725.0
Date:                Tue, 16 Aug 2022   Prob (F-statistic):           1.66e-91
Time:                        01:15:13   Log-Likelihood:                -1175.5
No. Observations:                 398   AIC:                             2355.
Df Residuals:                     396   BIC:                             2363.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         35.1748      0.492     71.519      0.000      34.208      36.142
x1            -0.0603      0.002    -26.926      0.000      -0.065      -0.056
==============================================================================
Omnibus:                       41.373   Durbin-Watson:                   1.394
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               60.024
Skew:                           0.711   Prob(JB):                     9.24e-14
Kurtosis:                       4.264   Cond. No.                         463.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The table above tells us that the intercept is 35.2 and the slope is -0.06.
<pre>
	     coef	  std err	      P>|t|	[0.025	0.975]
const	35.1748	0.492	 71.519	0.000	34.208	36.142
x1   	-0.0603	0.002	-26.926	0.000	-0.065	-0.056

</pre>

We'll talk more about what the entire table means later.

# Making the Design Matrix

In what follows we're going to do the math of OLS. You may remember this from statistics. If you don't recall, you can just skim over this section. The point is to show where the calculations of the slope and intercept actually come from.

In [ ]:
X = data[ ['displacement'] ]
y = data[ ['mpg'] ]

In [ ]:
X.head()

,displacement
0,307.0
1,350.0
2,318.0
3,304.0
4,302.0


Adding the constant term:

In [ ]:
data["constant"] = 1
X = data[['displacement', "constant"]]

In [ ]:
X.head()

,displacement,constant
0,307.0,1
1,350.0,1
2,318.0,1
3,304.0,1
4,302.0,1


This $(X^T X)^{-1} X^T$ is called a projection matrix. The idea is that we're going to project the values of y onto the space spanned by x. (If you're not familiar with matrix algeabra that's fine. Just skip over this part.) Even if you don't understand the math, watch how we can calculate the slope and the intercept terms using this equation.

$$
(X^T X)^{-1} X^T y
$$

In [ ]:
X.T @ X

,displacement,constant
displacement,19206864.25,76983.5
constant,76983.50,398.0


In [ ]:
X.T @ y

,mpg
displacement,1550039.4
constant,9358.8


## Solving the Linear System

In [ ]:
from numpy.linalg import inv, solve

The exact math from lecture.

$$
(X^T X)^{-1} X^T y
$$

In [ ]:
inv(X.T @ X) @ (X.T @ y)

,mpg
0,-0.060282
1,35.174750


More numerically stable and computationally efficient.

$$
A\theta = b
$$

$$
X^T X \theta = X^T y
$$

In [ ]:
solve(X.T @ X, X.T @ y)

array([[-0.06028241],
       [35.17475015]])

Boom! We did it! A little mathematical work and we were able to reproduce the answer from the OLS regression trendline above. Cool! 

But, aside from the fact that you can now impress your friends at a party, this is rather cumbersome especially as we add more dimensions. Let's find a nice software package that will do this for us.

## Using a software package:

sklearn is a software package for machine learning in python.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Let's see the results with the X and y values we assigned above.
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
print(model.coef_)

[[-0.06028241 35.17475015]]


## Making Predictions

In [ ]:
theta = solve(X.T @ X, X.T @ y)

In [ ]:
theta

array([[-0.06028241],
       [35.17475015]])

### Making a simple prediction
We can make a simple prediction using the slope and intercept we calculated. Recall, our equation for a line is equal to y = b + mx. Since we know the values for X, b and m (the intercept and slope terms) it is easy to estimate y. We call the estimate **yhat** or $\hat{y}$.

In [ ]:
data['yhat'] = (X @ theta)  # matrix multiplication -- again, just to show you that it works. We'll use a library to do this.
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,constant,yhat
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,1,16.668052
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,1,14.075908
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,1,16.004945
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,1,16.848899
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,1,16.969464


Now let's try using the predict() function that is part of the sklearn model we just esimated, above.

In [ ]:

data['yhat'] = model.predict(X)
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,constant,yhat
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,1,16.668052
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,1,14.075908
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,1,16.004945
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,1,16.848899
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,1,16.969464


The estimated y values (yhat) are the same!

## Examining the Residuals

Recall that the residuals are the difference between the actual value of **y** and our predicted value **yhat** or $\hat{y}$.

In [ ]:
data["residual"] = data["mpg"] - data["yhat"] 

The residuals are all that is left over from our regression once we attempt to explain the variation of y using x. 


In [ ]:
fig = px.scatter(data, x="displacement", y="residual")
fig.add_trace(go.Scatter(x=[50, 475], y=[0,0], name = "Model"))

In [ ]:
# Note that the sum of residuals is zero.
data['residual'].sum()

-5.6274984672199935e-12

## Root Mean Squared Error

The RMSE is a measure we can use to determine how well our model predicts y.

For every observation, take the difference between y and $\hat{y}$ and square the result. Squaring the term will exaggerate any large difference between the actual and predicted value. This will dramatically increase our error measure!

Then we divide by **n** the number of observations to get the average squared error. We then take the square root to return the units of the error measure to the same as y.

$$
\sqrt{\frac{1}{n} \sum_{i=1}^n  \left(y_i - \hat{y}_i\right)^2 } 
$$

In [ ]:
# RMSE calculated "by hand"
np.sqrt(np.mean(data['residual']**2))

4.6396293441245815

In [ ]:
# An alternative error measure is to take the mean of the absolute values of the errors (MAE).
data["residual"].abs().mean()

3.526327374875379

## Improving the Model

Our initial model wasn't perfect. Let's try to explain more of the variation in y by using more features (explanatory variables).

In [ ]:
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,constant,yhat,residual
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,1,16.668052,1.331948
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,1,14.075908,0.924092
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,1,16.004945,1.995055
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,1,16.848899,-0.848899
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,1,16.969464,0.030536


In [ ]:
# Add more features to a new model
model2 = LinearRegression(fit_intercept=True)
features = ["cylinders", "displacement", "weight", "model_year", "acceleration"]

model2.fit(data[features], data[['mpg']])
print(model2.coef_)

[[-0.25858516  0.00726771 -0.00692571  0.75530084  0.08034746]]


Get the predictions from our second model.

In [ ]:
data['yhat2'] = model2.predict(data[features])

What can we say about the magnitudes of the weights?

In [ ]:
px.histogram(data, 
             x = ["cylinders", "displacement", "weight"],
             barmode="overlay",
             marginal="box")

Well, our different features are measured on different scales. The number of cylinders goes from 4-8 while the weight of the car is more than 1,000 pounds.

In an OLS regression, this doesn't matter. But we'll see other methods where we need to ensure that all of our features are measured on the same scale. 

How would we do that?

### Rescaling the features
The answer is that we want to rescale the features. We don't lose any information we just change the scale. (Besides, we can always reverse the calculation to interpret any coefficients.) Moreover, for the most part, machine learning models are concerned with predicting the value of $\hat{y}$ not interpreting coefficients.

In [ ]:
# Thankfully, sklearn has a scaler built-in that we can use. We'll talk more about scaling 
# in a future lesson.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Z = scaler.fit_transform(data[features])

In [ ]:
Z

array([[ 1.49819126,  1.0906037 ,  0.63086987, -1.62742629, -1.29549834],
       [ 1.49819126,  1.5035143 ,  0.85433297, -1.62742629, -1.47703779],
       [ 1.49819126,  1.19623199,  0.55047045, -1.62742629, -1.65857724],
       ...,
       [-0.85632057, -0.56103873, -0.79858454,  1.62198339, -1.4407299 ],
       [-0.85632057, -0.70507731, -0.40841088,  1.62198339,  1.10082237],
       [-0.85632057, -0.71467988, -0.29608816,  1.62198339,  1.39128549]])

In [ ]:
# Now let's look at the scaled values. 
px.histogram(x = [Z[:,0], Z[:,1], Z[:,2]],
             barmode="overlay")

In [ ]:
model3 = LinearRegression(fit_intercept=True)
model3.fit(Z, data[['mpg']])
print(model3.coef_)
data['yhat3'] = model3.predict(Z)

[[-0.43930153  0.75684991 -5.85760433  2.78930976  0.22129477]]


In [ ]:
features

['cylinders', 'displacement', 'weight', 'model_year', 'acceleration']

In [ ]:
data

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,constant,yhat,residual,yhat2,yhat3
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,1,16.668052,1.331948,15.160369,15.160369
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,1,14.075908,0.924092,14.123749,14.123749
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,1,16.004945,1.995055,15.630915,15.630915
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,1,16.848899,-0.848899,15.630291,15.630291
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,1,16.969464,0.030536,15.384423,15.384423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl,1,26.735213,0.264787,29.278818,29.278818
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup,1,29.327357,14.672643,34.260400,34.260400
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage,1,27.036625,4.963375,32.349314,32.349314
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger,1,27.940861,0.059139,30.517248,30.517248


### Residual Analysis

Calculate the residuals from the second model.

In [ ]:
data['residual2'] = data['mpg'] - data['yhat2']

Now, calculate the RMSE. Looks like it has improved by adding more features!

In [ ]:
np.sqrt(np.mean(data['residual2']**2))

3.4143567605602256

## Computing [$R^2$](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score-the-coefficient-of-determination)

Now let's calculate the $R^2$ value. $R^2$ is a measure of the variation of y explained by X (i.e., all of our explanatory variables/features.)

$R^2$ values are between 0 and 1. (There is an adjusted $R^2$ value that can exceed these bounds slightly in some cases. We will talk about that later.)


In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(data["mpg"], data["yhat"])

0.646742183425786

Compare the $R^2$ of the first model to that of the second model:

In [ ]:
r2_score(data["mpg"], data["yhat2"])

0.8086876515237436

In [ ]:
import statsmodels.formula.api as smf
model = smf.ols( formula="mpg ~ displacement" , data = data)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.647
Model:                            OLS   Adj. R-squared:                  0.646
Method:                 Least Squares   F-statistic:                     725.0
Date:                Tue, 16 Aug 2022   Prob (F-statistic):           1.66e-91
Time:                        04:30:17   Log-Likelihood:                -1175.5
No. Observations:                 398   AIC:                             2355.
Df Residuals:                     396   BIC:                             2363.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       35.1748      0.492     71.519   

In [ ]:
#@title
# We use statsmodels library to print out the results nicely.
# There are two ways to use it.
import statsmodels.api as sm
data = sns.load_dataset('mpg')
data = data.dropna()
X = data.iloc[:, 1:-3]
y = data.iloc[:, 0]
X = sm.add_constant(X)
m = sm.OLS(y, X).fit() # here we fit our model using our x and y data defined above
print(m.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.708
Model:                            OLS   Adj. R-squared:                  0.704
Method:                 Least Squares   F-statistic:                     186.9
Date:                Tue, 16 Aug 2022   Prob (F-statistic):          9.82e-101
Time:                        04:31:02   Log-Likelihood:                -1120.1
No. Observations:                 392   AIC:                             2252.
Df Residuals:                     386   BIC:                             2276.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           46.2643      2.669     17.331   

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only

